In [ ]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import time

In [ ]:


# Read the CSV from sample_data folder
block_mapping = pd.read_csv('/content/eth_usdc_currentpositions.csv')
block_mapping['Datetime'] = pd.to_datetime(block_mapping['Datetime'])


# View the first few rows
block_mapping.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

# Generate dates - ADD AS MANY AS YOU WANT HERE
dates_to_plot = [
    '2022-02-01',
    '2022-02-07',
    '2022-02-13',
    '2022-02-19',
    '2022-02-25',
    '2022-02-24',
    '2022-02-26',
    '2022-02-27',
    '2022-02-28',
    '2022-03-01'
]

# The Graph API setup
uniswap_api_key = "2b45e1f034a61c2f41bf4bcd6731335a"
subgraph_id = "5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"
url = f"https://gateway.thegraph.com/api/{uniswap_api_key}/subgraphs/id/{subgraph_id}"

# Query template with block number
query = """
query GetPositions($poolId: String!, $blockNumber: Int!) {
  positions(
    where: { pool: $poolId, liquidity_gt: 0 }
    block: { number: $blockNumber }
    first: 1000
  ) {
    liquidity
    depositedToken0
    depositedToken1
    withdrawnToken0
    withdrawnToken1
    tickLower {
      tickIdx
    }
    tickUpper {
      tickIdx
    }
  }
}
"""

# Calculate grid dimensions based on number of dates
num_dates = len(dates_to_plot)
num_cols = 3  # Keep 3 columns
num_rows = int(np.ceil(num_dates / num_cols))  # Calculate needed rows

# Create figure with dynamic subplot grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 5 * num_rows))
axes = axes.flatten()  # Flatten to make indexing easier

# Tick range settings (adjust based on expected ETH price range)
tick_min = 150000
tick_max = 225000
bin_width = 600

for idx, target_date in enumerate(dates_to_plot):
    print(f"\nProcessing {target_date}...")

    # Get block number for this date
    matching_blocks = block_mapping[block_mapping['Datetime'] == target_date]
    if len(matching_blocks) == 0:
        print(f"  ⚠ No block found for {target_date}, skipping...")
        axes[idx].text(0.5, 0.5, f'No data for {target_date}',
                      ha='center', va='center', transform=axes[idx].transAxes)
        axes[idx].set_title(f"{target_date}\nNo Data")
        continue

    block_number = int(matching_blocks['Ethereum Block Number'].values[0])
    print(f"  Block number: {block_number}")

    # Query The Graph
    variables = {
        "poolId": "0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640".lower(),
        "blockNumber": block_number
    }

    response = requests.post(
        url,
        json={"query": query, "variables": variables},
        headers={"Content-Type": "application/json"}
    )
    data = response.json()

    # Check for errors
    if 'errors' in data:
        print(f"  ⚠ Error: {data['errors']}")
        axes[idx].text(0.5, 0.5, f'Query error for {target_date}',
                      ha='center', va='center', transform=axes[idx].transAxes)
        axes[idx].set_title(f"{target_date}\nError")
        continue

    # Convert to dataframe
    positions_df = pd.DataFrame(columns=[
        'Deposited Token 0', 'Deposited Token 1', 'Total Liquidity Deposited',
        'Lower Tick', 'Upper Tick', 'Withdrawn Token 0', 'Withdrawn Token 1'
    ])

    for position_idx in range(len(data['data']['positions'])):
        positions_df.loc[len(positions_df)] = [
            data['data']['positions'][position_idx]['depositedToken0'],
            data['data']['positions'][position_idx]['depositedToken1'],
            data['data']['positions'][position_idx]['liquidity'],
            data['data']['positions'][position_idx]['tickLower']['tickIdx'],
            data['data']['positions'][position_idx]['tickUpper']['tickIdx'],
            data['data']['positions'][position_idx]['withdrawnToken0'],
            data['data']['positions'][position_idx]['withdrawnToken1']
        ]

    cols_to_convert = [col for col in positions_df.columns]
    positions_df[cols_to_convert] = positions_df[cols_to_convert].astype(float)

    print(f"  Found {len(positions_df)} active positions")

    # Create liquidity distribution bins
    bins = np.arange(tick_min, tick_max + bin_width, bin_width)
    liquidity_per_bin = np.zeros(len(bins) - 1)

    for _, row in positions_df.iterrows():
        bin_indices = np.where(
            (bins[:-1] < row['Upper Tick']) & (bins[1:] > row['Lower Tick'])
        )[0]
        liquidity_per_bin[bin_indices] += row['Total Liquidity Deposited']

    bin_centers = (bins[:-1] + bins[1:]) / 2

    # Plot on subplot
    axes[idx].bar(bin_centers, liquidity_per_bin, width=bin_width,
                  edgecolor='black', alpha=0.7, color='steelblue')
    axes[idx].set_title(f"{target_date}\nBlock: {block_number}", fontsize=12, fontweight='bold')
    axes[idx].set_xlabel("Tick Index", fontsize=10)
    axes[idx].set_ylabel("Liquidity", fontsize=10)
    axes[idx].grid(axis='y', alpha=0.3)
    axes[idx].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

    # Add total liquidity annotation
    total_liq = liquidity_per_bin.sum()
    axes[idx].text(0.95, 0.95, f'Total: {total_liq:.2e}',
                  transform=axes[idx].transAxes,
                  ha='right', va='top',
                  bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                  fontsize=9)

# Hide any unused subplots
for idx in range(num_dates, len(axes)):
    fig.delaxes(axes[idx])

plt.suptitle('ETH/USDC Liquidity Distribution - Feb & Early March 2022',
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print(f"\n✅ All {num_dates} graphs generated!")